In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('train_full.csv')

In [36]:
df

,keyword,location,text,target
0,demolished,NaN,Just demolished a Snowball ??,0
1,loud%20bang,NaN,I don't laugh out loud at many things. But man...,0
2,drought,San Francisco ; CA,ItÛªs time to do away with drought.Check out ...,1
3,apocalypse,Harlingen; TX,My niece just asked me 'would you be scared if...,0
4,bridge%20collapse,Mumbai ; India,Warne shocked over Australia's epic collapse a...,1
...,...,...,...,...
996,ablaze,San Francisco,what time does your talk go until? I don't kno...,0
997,ruin,London / Birmingham,Im so anxious though because so many ppl will ...,0
998,whirlwind,pettyville; usa,this week has been a whirlwind but this is ex...,0
999,rescue,Toronto; Ontario,UD: Rescue (Structural Collapse) - Scott Road ...,1


In [57]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle 

# Загрузка датасета из файла
df = pd.read_csv('train_full.csv')

# Создание списка признаков на основе текста сообщения
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

# Определение целевой переменной
y = df['target']

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)

# Оценка качества модели на тестовой выборке
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# Save to file in the current working directory 
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'wb') as file: 
        pickle.dump(model, file) 
 
 # Load from file 
with open(pkl_filename, 'rb') as file: 
    pickle_model = pickle.load(file) 
 
 # Calculate the accuracy score and predict target values 
score = pickle_model.score(X_test, y_test) 
print("Test score: {0:.2f} %".format(100 * score)) 
Ypredict = pickle_model.predict(X_test) 

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

Accuracy: 0.7263681592039801
Test score: 72.64 %


In [48]:
import gensim
from gensim import corpora

# создание словаря для корпуса
texts = []
for keyword in df['keyword']:
    if isinstance(keyword, str):
        words = keyword.lower().split()
        texts.append(words)
dictionary = corpora.Dictionary(texts)

# создание корпуса
corpus = []
for keyword in df['keyword']:
    if isinstance(keyword, str):
        words = keyword.lower().split()
        corpus.append(dictionary.doc2bow(words))

# создание тематической модели
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# вывод топ-5 слов для каждой темы
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.088*"collided" + 0.088*"wreck" + 0.079*"collision" + 0.062*"burning" + 0.062*"wild%20fires" + 0.062*"damage" + 0.053*"structural%20failure" + 0.053*"accident" + 0.053*"obliteration" + 0.044*"hazard"
Topic: 1 
Words: 0.096*"body%20bag" + 0.084*"burned" + 0.084*"wrecked" + 0.072*"thunderstorm" + 0.072*"crushed" + 0.072*"armageddon" + 0.060*"devastated" + 0.048*"hijacker" + 0.048*"injured" + 0.048*"siren"
Topic: 2 
Words: 0.100*"hail" + 0.084*"derailed" + 0.084*"upheaval" + 0.075*"sinkhole" + 0.067*"windstorm" + 0.059*"obliterated" + 0.059*"refugees" + 0.051*"whirlwind" + 0.050*"injury" + 0.050*"emergency%20services"
Topic: 3 
Words: 0.101*"tornado" + 0.079*"hijacking" + 0.079*"rescued" + 0.079*"dust%20storm" + 0.068*"landslide" + 0.068*"rescue" + 0.057*"nuclear%20disaster" + 0.057*"army" + 0.046*"blaze" + 0.035*"fear"
Topic: 4 
Words: 0.073*"flattened" + 0.058*"demolition" + 0.051*"blight" + 0.044*"curfew" + 0.044*"tsunami" + 0.044*"thunder" + 0.044*"screaming" + 0.044

In [50]:
import re

# вывод топ-10 слов для каждой темы
for idx, topic in lda_model.print_topics(-1):
    words = re.sub(r'[^a-zA-Z\s]', '', topic).split()
    words = [word for word in words if len(word) > 1]  # удалить однобуквенные слова
    print('Topic: {} \nWords: {}'.format(idx, ' '.join(words)))

Topic: 0 
Words: collided wreck collision burning wildfires damage structuralfailure accident obliteration hazard
Topic: 1 
Words: bodybag burned wrecked thunderstorm crushed armageddon devastated hijacker injured siren
Topic: 2 
Words: hail derailed upheaval sinkhole windstorm obliterated refugees whirlwind injury emergencyservices
Topic: 3 
Words: tornado hijacking rescued duststorm landslide rescue nucleardisaster army blaze fear
Topic: 4 
Words: flattened demolition blight curfew tsunami thunder screaming drowned forestfires blazing
Topic: 5 
Words: hazardous razed trouble oilspill panic inundated firstresponders deluge collapsed snowstorm
Topic: 6 
Words: collide emergency wreckage drowning crashed trapped apocalypse engulfed quarantine attack
Topic: 7 
Words: mudslide debris sandstorm storm ruin derailment explode collapse sunk heatwave
Topic: 8 
Words: meltdown evacuation drought injuries traumatised exploded desolation wildfire emergencyplan desolate
Topic: 9 
Words: bridgecoll